# Casos prácticos

En este notebook vamos a abordar dos casos prácticos interesantes.

1. Predicción (*forecasting*) de la demanda de bicicletas
2. Clasificación multiclase de imágenes

## Librerías y funciones

Lo primero es cargar las librerías y funciones necesarias.

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline

cm = plt.cm.RdBu
cm_bright = ListedColormap(['#FF0000', '#0000FF'])

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def plot_confusion_matrix(confmat):
    fig, ax = plt.subplots(figsize=(7, 7))
    ax.matshow(confmat, cmap=plt.cm.Blues, alpha=0.5)
    for i in range(confmat.shape[0]):
        for j in range(confmat.shape[1]):
            ax.text(x=j, y=i, s=confmat[i, j], va='center', ha='center')

    plt.xlabel('predicted label')
    plt.ylabel('true label')

    plt.tight_layout()
    plt.show()

# 1. Predicción de la demanda de bicicletas

El problema está descrito [aquí](https://christophm.github.io/interpretable-ml-book/bike-data.html), y los datos pueden descargarse en la [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset), aunque los hemos modificado un poco para hacer el problema más parecido al [*challenge*](https://www.kaggle.com/c/bike-sharing-demand/data) original.

In [ ]:
data = pd.read_csv('./data/bikes.csv',sep=';', decimal='.')
data.head()

<div class = "alert alert-success">
**EJERCICIO**: Interprete los datos y realiza tus primeras hipótesis sobre qué variables son de interés
</div>

<div class = "alert alert-success">
**EJERCICIO**: Elimine las columnas *instant*, *casual* y *registered*
</div>

In [ ]:
# your code here


## 1.1 *Feature Engineering*

Vamos a trabajar con las fechas para crear algunas variables auxiliares

In [ ]:
from datetime import datetime

data['dteday'] = data['dteday'].apply(lambda x: datetime.strptime(x,'%d-%m-%Y'))
data.head()

In [ ]:
data['year'] = data['dteday'].apply(lambda x: x.year - 2011)
data.head()

In [ ]:
data['month'] = data['dteday'].apply(lambda x: x.month)
data.head()

In [ ]:
data['weekday'] = data['dteday'].apply(lambda x: x.isoweekday())
data.head()

Llegados a este punto, podemos eliminar la variable *dteday*

In [ ]:
data = data.drop(['dteday'],axis=1)
data.head()

## 1.2 Análisis de correlación

<div class = "alert alert-success">
**EJERCICIO**: Represente la variable *temp* vs *atemp*.
</div>

In [ ]:
# your code here


A la vista está que son variables altamente correlacionadas. Podemos eliminar *temp*, ya que nos afecta más la sensación térmica que la temperatura real.

In [ ]:
data = data.drop(['temp'],axis=1)
data.head()

<div class = "alert alert-success">
**EJERCICIO**(@home): Realice un análisis exploratorio exhaustivo.
</div>

# 1.3 Codificación de variables categóricas

Tenemos varias variables categóricas: *season*, *weathersit*, *month* y *weekday*. Cuando trabajamos con series temporales, es común crear variables *dummies* asociadas a cada una de las situaciones de las variables categóricas. Para ello, usamos get_dummies, que es básicamente el onehot de Pandas:

- [pd.get_dummies()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html): se aplica directamente sobre el dataframe puede [utilizarse](https://towardsdatascience.com/the-dummys-guide-to-creating-dummy-variables-f21faddb1d40) con strings directamente. La pega es que genera un nuevo dataframe que hay que agregar al original.

La codificación "dummy" de variables categóricas en problemas de clasificación/regresión es en general una mala idea, como vimos anteriormente, porque aumenta mucho la dimensionalidad de entrada; en series temporales resulta adecuado para explicar el efecto de una situación temporal en la variable target.

<div class = "alert alert-success">
**EJERCICIO**(@home): Prueba no realizar esta codificación "dummy" y entrena un modelo de machine learning para predecir la demanda de bicicletas.
</div>

Comenzamos por la variable *season*. Veamos qué hace *get_dummies()*.

In [ ]:
pd.get_dummies(data['season'], prefix = 'season')

In [ ]:
dummy = pd.get_dummies(data['season'], prefix = 'season')

data = pd.concat([data,dummy],axis=1).drop(['season'],axis=1)
data.head()

## 1.4 División train/test 

In [ ]:
# preparamos los datos
features = data.columns.drop(['cnt'])
X = data[features].values
y = data['cnt'].values

print('Filas, columnas', X.shape)

In [ ]:
# Paso 1:
offset = 182 # 0.25 of 731

X_train = X[:-offset, :]
y_train = y[:-offset]
X_test  = X[-offset:, :]
y_test  = y[-offset:]

plt.plot(range(0,len(y_train)),y_train, label='train')
plt.plot(range(len(y_train),len(y)),y_test,label='test')
plt.legend()
plt.show()

## 1.5 Búsqueda de parámetros libres

In [ ]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.linear_model import Lasso

tscv = TimeSeriesSplit(n_splits=3)

alpha_vector = np.logspace(-4,4,20)
param_grid = {'alpha': alpha_vector}

grid = GridSearchCV(Lasso(), param_grid=param_grid, cv = tscv.split(X_train)).fit(X_train, y_train)

In [ ]:
print("best mean cross-validation score: {:.3f}".format(grid.best_score_))
print("best parameters: {}".format(grid.best_params_))

scores = grid.cv_results_['mean_test_score']
std_scores = grid.cv_results_['std_test_score']
plt.errorbar(np.log10(alpha_vector),scores,yerr=std_scores, fmt='o-',ecolor='g')
plt.xlabel('log(alpha)',fontsize=16)
plt.ylabel('CV MSE')
plt.grid()
plt.show()

## 1.6 Métricas en test

In [ ]:
from sklearn.metrics import mean_squared_error

alpha_optimo = grid.best_params_['alpha']
lasso = Lasso(alpha = alpha_optimo).fit(X_train,y_train)

ytrainLasso = lasso.predict(X_train)
ytestLasso  = lasso.predict(X_test)

mseTrainModelLasso = mean_squared_error(y_train,ytrainLasso)
mseTestModelLasso = mean_squared_error(y_test,ytestLasso)

print('MSE Modelo Lasso (train): %0.3g' % mseTrainModelLasso)
print('MSE Modelo Lasso (test) : %0.3g' % mseTestModelLasso)

w = lasso.coef_
for f,wi in zip(features,w):
    print(f,wi)

<div class = "alert alert-success">
**EJERCICIO**: Representa la predicción obtenida junto con la serie real (train+test)
</div>

In [ ]:
# your code here


# 2. Clasificación multiclase de imágenes

En este caso vamos a utilizar la famosa base de datos de [MNIST](http://yann.lecun.com/exdb/mnist/). Esta base de datos contiene

* Training set: train-images-idx3-ubyte.gz (9.9 MB, 47 MB unzipped, 60,000 samples)
* Training set labels: train-labels-idx1-ubyte.gz (29 KB, 60 KB unzipped, 60,000 labels)
* Test set images: t10k-images-idx3-ubyte.gz (1.6 MB, 7.8 MB, 10,000 samples)
* Test set labels: t10k-labels-idx1-ubyte.gz (5 KB, 10 KB unzipped, 10,000 labels)

Estas imágenes se pueden descargar a partir del siguiente código (previamente hay que descargarse los archivos).

In [ ]:
import os
import struct
import numpy as np
import gzip
 
def load_mnist(path, kind='train'):
    """Load MNIST data from `path`"""
    labels_path = os.path.join(path, 
                               '%s-labels-idx1-ubyte.gz' % kind)
    images_path = os.path.join(path, 
                               '%s-images-idx3-ubyte.gz' % kind)
        
    with gzip.open(labels_path, 'rb') as lbpath:
        lbpath.read(8)
        buffer = lbpath.read()
        labels = np.frombuffer(buffer, dtype=np.uint8)

    with gzip.open(images_path, 'rb') as imgpath:
        imgpath.read(16)
        buffer = imgpath.read()
        images = np.frombuffer(buffer, 
                               dtype=np.uint8).reshape(
            len(labels), 784).astype(np.float64)
 
    return images, labels

In [ ]:
X_train, y_train = load_mnist('./data/mnist/', kind='train')
print('Rows: %d, columns: %d' % (X_train.shape[0], X_train.shape[1]))

In [ ]:
X_test, y_test = load_mnist('mnist/', kind='t10k')
print('Rows: %d, columns: %d' % (X_test.shape[0], X_test.shape[1]))

¡Fíjate que el conjunto de entrenamiento son los pixels de la imagen tal cual!

In [ ]:
print ("X train shape: ", X_train.shape)
print ("y train shape: ", y_train.shape)
print ("X test shape: ",  X_test.shape)
print ("y test shape: ", y_test.shape)

<div class = "alert alert-success">
**EJERCICIO**: Entrena un modelo de regresión logística con C = 10 y calcula sus prestaciones en el conjunto de test. A lo mejor te resulta de utilidad revisar la [documentación](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).
</div>

In [ ]:
from sklearn.linear_model import LogisticRegression

# your code here


<div class = "alert alert-success">
**EJERCICIO**: Calcula y representa la matriz de confusión, ¿qué conclusiones puedes sacar?
</div>

In [ ]:
from sklearn.metrics import confusion_matrix

# your code here


# Pequeña ayuda para la práctica final

## 1. División

El método `train_test_split` es mucho más potente que lo que hemos visto en clase, y permite hacer muchas más cosas. En clase siempre hemos particionado en cuatro (xtrain, xtest, ytrain, ytest) obteniendo arrays de numpy, pero no es la única opción. Aquí tenéis un único fichero .csv, y podéis usar la función para obtener dos subconjuntos: train y test.

In [ ]:
from sklearn.model_selection import train_test_split

full_df = pd.read_csv('./airbnb-listings-extract.csv', sep=';', decimal='.')
train, test = train_test_split(full_df, test_size=0.2, shuffle=True, random_state=0)

print(f'Dimensiones del dataset de training: {train.shape}')
print(f'Dimensiones del dataset de test: {test.shape}')

# Guardamos
train.to_csv('./train.csv', sep=';', decimal='.', index=False)
test.to_csv('./test.csv', sep=';', decimal='.', index=False)

# A partir de este momento cargamos el dataset de train y trabajamos ÚNICAMENTE con él. 

df = pd.read_csv('./train.csv', sep=';', decimal='.')

# etc...

A la hora de dividir en ``X`` e ``y`` lo que tendréis que hacer es algo como esto:

In [ ]:
from sklearn import preprocessing
data = df.values
y_train = data[:,0:1]     # nos quedamos con la 1ª columna, price
X_train = data[:,1:]      # nos quedamos con el resto
feature_names = df.columns[1:]
# Escalamos (con los datos de train)
scaler = preprocessing.StandardScaler().fit(X_train)
XtrainScaled = scaler.transform(X_train)

Y cuando queráis ya evaluar el modelo (después de hacer CV y demás) tendréis que cargar el csv de test:

In [ ]:
df_test = pd.read_csv('./test.csv', sep=';', decimal='.')

# aplicarle LAS MISMAS transformaciones que hayáis hecho en train 
#(drop de columnas, filtros, generaciones, fill...) y sacar los datos:

data_test = df_test.values
y_test = data_test[:,0:1]     # nos quedamos con la 1ª columna, price
X_test = data_test[:,1:]      # nos quedamos con el resto
feature_names_test = df_test.columns[1:]

# recordad que esta normalización/escalado la realizo con el scaler anterior, basado en los datos de training!
XtestScaled = scaler.transform(X_test) 

## 2. Imputación

Todos los datasets que hemos usado están preprocesados mínimamente: no hay valores ausentes, pocos outliers... en la realidad y en la práctica final no va a ser así. El tratamiento de outliers es un poco artesanal: se puede hacer con un filtro, por ejemplo, como ya hemos visto, o analizando manualmente los scatter plots.

Para los valores ausentes, hay que imputar. Es muy sencillo con pandas, usando fillna:

`df["Feature"].fillna(df["Feature"].mode()[0], inplace=True)`

El caso de arriba rellena con la moda (valor más frecuente), en otras ocasiones es preferible usar la media, y en algunos (pocos) casos se puede hacer con ceros.

## 3. Codificación de variables categóricas

[MeanEncoder](https://maxhalford.github.io/blog/target-encoding/): no está implementado (que yo sepa) en scikit learn, pero da igual. Asigna un valor a cada variable categórica según la media de la columna objetivo para el conjunto de registros que tienen esa variable categórica. Es decir, si quisiera categorizar la variable "Barrio" con un ME, lo que tendría que hacer es calcular la media de precio en cada barrio (Villaverde, Chamberí, etc) y sustituir el nombre del barrio por esa media. Ojito con el data leakage, os dejo un ejemplo debajo de cómo hacerlo bien.

Tutoriales sobre codificación de variables categóricas: [Tutorial 1](https://towardsdatascience.com/encoding-categorical-features-21a2651a065c), [tutorial 2](https://towardsdatascience.com/all-about-categorical-variable-encoding-305f3361fd02)

In [ ]:
categorical = ['cat1', 'cat2', 'cat3']

mean_map = {}
for c in categorical:
    mean = data.groupby(c)['target'].mean()
    data[c] = data[c].map(mean)    
    mean_map[c] = mean

# Si hubiera test, luego se haría:
#for c in categorical:
#    data_test[c] = data_test[c].map(mean_map[c])

data.head()

Este es uno de los tutoriales más completos que he visto: [Encoding done the right way](https://maxhalford.github.io/blog/target-encoding/) y me hace especial gracia porque dice esto: "Label encoding is useless and you should never use it". No estoy 100% de acuerdo obviamente :) pero sí que es cierto que cuando hay muchas categorías (es decir, no es binario) un LE puede llevar a errores porque asigna números a cada una de ellas, con lo cual el algoritmo puede "aprender" erróneamente. Supongamos que tengo una categoría barrio que quiero categorizar:

- Barrio céntrico moderno y caro -> LE -> 1
- Otro barrio -> LE -> 2
- Otro más -> LE -> 3
- Y otro -> LE -> 4
- Barrio periférico y obrero -> LE -> 5

Mis categorías tras el LE pasarían a ser 1-5, pero qué quiere decir esto? Que 5 es mayor que 1? Que 3 es menor que 5? No, porque no existe esa relación entre los barrios, pero SÍ entre los números! Entonces el algoritmo podría decidir que de alguna manera "Barrio periférico y obrero > Barrio céntrico moderno y caro" porque 5 > 1.